In [34]:
import pandas as pd
from utils import clean_text, combine_word2idx
import re
import numpy as np
import json
from tqdm import tqdm_notebook
from collections import Counter

In [22]:
all_data = pd.read_csv('all_milk_with_all_field.csv')

/root/anaconda2/envs/tfp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
label_data = pd.read_csv('./total_purchase_text - totalpurchase_text.csv')

In [24]:
all_data = pd.concat([all_data['mention'], label_data['Mention']], axis=0)

In [26]:
all_data.dropna(inplace=True)

In [27]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [28]:
strip_data = [strip_tags(x) for x in tqdm_notebook(all_data)]

In [29]:
data = [clean_text(x.strip()) for x in tqdm_notebook(strip_data)]

In [31]:
with open('../102/word2idx.json', 'r') as inp:
    old_w2i = json.load(inp)
with open('../102/char2idx.json', 'r') as inp:
    old_c2i = json.load(inp)

In [35]:
counter = Counter()

In [38]:
counter.update(w for x in tqdm_notebook(data) for w in x.split())

In [40]:
idx2word = [w for w, c in counter.most_common() if c >= 10]

In [43]:
new_w2i = combine_word2idx(old_w2i, idx2word)

In [46]:
char_count = Counter()

In [47]:
char_count.update(c for w in new_w2i for c in w)

In [50]:
new_char = [c for c in char_count if c not in old_c2i]

In [54]:
for w in new_w2i:
    if w != '<UNK>':
        new_w2i[w] += 1
    else:
        new_w2i[w] = 1

In [62]:
with open('word2idx.json', 'w') as out:
    json.dump(new_w2i, out)
with open('char2idx.json', 'w') as out:
    json.dump(new_c2i, out)

In [69]:
all_word = [new_w2i.get(w, new_w2i['<UNK>']) for s in data for w in s.split()]

In [71]:
word2char = {w: [new_c2i[c] for c in w] for w in new_w2i}

In [80]:
all_char = [word2char.get(w, word2char['<UNK>']) for s in data for w in s.split()]

In [73]:
idx2word = {i: w for w, i in new_w2i.items()}
idx2char = {i: w for w, i in new_c2i.items()}

In [84]:
len(all_char)

20402743

In [85]:
np.save('lm_train_word.npy', all_word[:20000000])
np.save('lm_train_char.npy', all_char[:20000000])

In [86]:
np.save('lm_valid_word.npy', all_word[20000000:])
np.save('lm_valid_char.npy', all_char[20000000:])